<a href="https://colab.research.google.com/github/stevereiner/llama-relik/blob/main/llama-relik-fastcoref.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Your Neo4j graph database info
# Recommended: using Neo4j Aura, which provides a free cloud instance that can easily be accessed from a Google Colab notebook
username="neo4j"
password="your password"
url="neo4j+s://xxxxxxxx.databases.neo4j.io"

import os
# Your OpenAI key
os.environ["OPENAI_API_KEY"] = "sk-your-key"

In [ ]:
!pip install --quiet fastcoref spacy
!pip install --quiet llama-index-extractors-relik llama-index-graph-stores-neo4j llama-index-llms-openai llama-index

In [ ]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from llama_index.graph_stores.neo4j import Neo4jPGStore

graph_store = Neo4jPGStore(
    username=username,
    password=password,
    url=url,
    refresh_schema=False
)

In [ ]:
import pandas as pd

NUMBER_OF_ARTICLES = 10
news = pd.read_csv("https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/news_articles.csv")
news = news.head(NUMBER_OF_ARTICLES)

In [ ]:
from fastcoref import spacy_component
import spacy
coref_nlp = spacy.load('en_core_web_lg')
coref_nlp.add_pipe('fastcoref')

text = 'Alice goes down the rabbit hole. Where she would discover a new reality beyond her expectations.'
doc = coref_nlp(text, component_cfg={"fastcoref": {'resolve_text': True}})
doc._.coref_clusters
print(doc._.resolved_text)


def coref_text(text):
    coref_doc = coref_nlp(text, component_cfg={"fastcoref": {'resolve_text': True}})
    resolved_text = coref_doc._.resolved_text
    return resolved_text

print(
    coref_text("Tomaz is so cool. He can solve various Python dependencies and not cry")
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

Alice goes down the rabbit hole. Where Alice would discover a new reality beyond Alice's expectations.


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

Tomaz is so cool. Tomaz can solve various Python dependencies and not cry


In [ ]:
from llama_index.core import Document
from datetime import datetime

news["coref_text"] = news["text"].apply(coref_text)
documents = [
    Document(text=f"{row['title']}: {row['coref_text']}") for i, row in news.iterrows()
]


print(coref_text(news['text'][5]))

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

Jude Bellingham has also insisted Jude Bellingham's move to Real Madrid had nothing to do with money, as Jude Bellingham explained how Jude Bellingham's transfer was wrapped up so quickly.
“Money is not a thing for Jude Bellingham,” Jude Bellingham said.
“Jude Bellingham dont think about money at all when Jude Bellingham make these kinds of decisions. Jude Bellingham never have and Jude Bellingham never will. Jude Bellingham play the game purely out of love.
“Jude Bellingham spoke with people from Real Madrid when Jude Bellingham was given permission by Borussia Dortmund and Jude Bellingham love the feeling Jude Bellingham got from Real Madrid. Jude Bellingham couldn’t hide the feeling I got from the club. Jude Bellingham told people from Real Madrid straight away what Jude Bellingham felt and after that happened on Monday it all happened quickly.”
Jude Bellingham also opened up on how Jude Bellingham was given the No.5 shirt, which was worn by defender Jesus Vallejo last season, and w

In [ ]:
from llama_index.extractors.relik.base import RelikPathExtractor

relik = RelikPathExtractor(
    model="relik-ie/relik-relation-extraction-small", model_config={"skip_metadata": True}
)

# Use on Pro Collab with GPU
# relik = RelikPathExtractor(
#    model="relik-ie/relik-cie-small", model_config={"skip_metadata": True, "device":"cuda"}
# )

                ___              __         
               /\_ \      __    /\ \        
 _ __     __   \//\ \    /\_\   \ \ \/'\    
/\`'__\ /'__`\   \ \ \   \/\ \   \ \ , <    
\ \ \/ /\  __/    \_\ \_  \ \ \   \ \ \\`\  
 \ \_\ \ \____\   /\____\  \ \_\   \ \_\ \_\
  \/_/  \/____/   \/____/   \/_/    \/_/\/_/
                                            
                                            



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4o", temperature=0.0)
embed_model = OpenAIEmbedding(model_name="text-embedding-3-small")

In [ ]:
from llama_index.core import PropertyGraphIndex

index = PropertyGraphIndex.from_documents(
    documents,
    kg_extractors=[relik],
    llm=llm,
    embed_model=embed_model,
    property_graph_store=graph_store,
    show_progress=True,
)

Parsing nodes:   0%|          | 0/10 [00:00<?, ?it/s]

Extracting triples:   0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

[2024-09-17 23:06:21,762] [INFO] [httpx._send_single_request:1786] [PID:24743] HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Generating embeddings:   0%|          | 0/6 [00:00<?, ?it/s]

[2024-09-17 23:06:22,172] [INFO] [httpx._send_single_request:1786] [PID:24743] HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
[2024-09-17 23:06:22,258] [INFO] [httpx._send_single_request:1786] [PID:24743] HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
[2024-09-17 23:06:22,264] [INFO] [httpx._send_single_request:1786] [PID:24743] HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
[2024-09-17 23:06:22,279] [INFO] [httpx._send_single_request:1786] [PID:24743] HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
[2024-09-17 23:06:22,320] [INFO] [httpx._send_single_request:1786] [PID:24743] HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Generating embeddings:  17%|█▋        | 1/6 [00:00<00:02,  2.11it/s]

[2024-09-17 23:06:22,387] [INFO] [httpx._send_single_request:1786] [PID:24743] HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Generating embeddings: 100%|██████████| 6/6 [00:00<00:00,  8.27it/s]


In [ ]:
query_engine = index.as_query_engine(include_text=True)

response = query_engine.query("What happened at Ryanair?")

print(str(response))

[2024-09-17 23:06:31,881] [INFO] [httpx._send_single_request:1786] [PID:24743] HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
[2024-09-17 23:06:32,596] [INFO] [httpx._send_single_request:1786] [PID:24743] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[2024-09-17 23:06:33,992] [INFO] [httpx._send_single_request:1038] [PID:24743] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
The chief pilot at Ryanair was dismissed due to allegations of sexual misconduct and harassment towards female colleagues.
